# Selenium test setup
This notebook goes through the selenium test setup.

With selenium, we can open up an entire browser, and have full javascript capabilities (something that we miss with the BeautifulSoup tests). We can click on links, press keys, etc.

> As with all jupyter notebooks, make sure that you run the cells in order!

# Setup

First import all the required packages

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options

from pathlib import Path
import yaml

import time
import os
import sys
import subprocess

Set an extra path, so we can import our module

In [ ]:
p = str(Path(globals()['_dh'][0]).parent)
print(p)
sys.path.insert(1, p)

# import our module
from tests.lib import *
 


From the test.lib module, we call the get_paths function to get all the relevant paths. 

> You can edit the settings in `obsidian-html/ci/tests/lib.py`, but the paths should all be relative to the git folder, so there is no need to edit them now.


In [ ]:
paths = get_paths()

print('\n'.join([f'{p[0]}\n{p[1].as_posix()}\n' for p in paths.items()]))

# Compile our website

compile our vault to markdown and html

In [ ]:
# Create temp dir
os.chdir(paths['root'])
paths['temp_dir'].mkdir(exist_ok=True)

# Compile output
customize_default_config([])
convert_vault()

We can run this compilation again with different settings, by overriding the default. Take this example:

In [ ]:
# clean temp dir
cleanup_temp_dir()

# custom settings
customize_default_config([
    ('toggles/features/create_index_from_tags/enabled', True),
    ('toggles/features/create_index_from_tags/tags', ['type/index1', 'type/index2'])
])
convert_vault()

# Start webserver
Run the webserver at port 8080 in the current directory

In [ ]:
webserver_process = subprocess.Popen(
    [
        'python', 
        '-m', 
        'http.server', 
        '--directory', 
        paths['html_output_folder'], 
        '8080'
    ], 
    stdout=subprocess.DEVNULL, 
    stderr=subprocess.DEVNULL
)

# Open selenium browser

> Note that if you uncomment the line `options.headless = True`, the browser will not open a screen that we can see. This is useful for running this script in a container.

In [ ]:
# add ci/src to path so that we can load the gecko driver
os.environ["PATH"] = paths['ci_src'].as_posix() + ':' + os.environ["PATH"]

# Configure options
options = Options()
#options.headless = True

# Start selenium
driver = webdriver.Firefox(options=options)

In [ ]:
# Load page
driver.get("http://localhost:8080/")

# Run tests

Get the first container, print the first h1, and get the first link in that container that isn't an anchor link, and click it.

In [ ]:
level = 0

In [ ]:
# get container
container = driver.find_element(by=By.ID, value=f'level-{level}')

# print first h1
try:
    h1 = container.find_element(by=By.TAG_NAME, value='h1')
    print(h1.text)
except:
    print("h1 not found")

# click first non-anchor/backlink link
links = container.find_elements(by=By.TAG_NAME, value='a')
for link in links:
    if exclude_str(['anchor','backlink'], link.get_attribute('class')):
        link.click()
        
        # target the new note next
        level += 1
        
        # cancel execution (don't open all links on a note)
        break
        


Now, because level has incremented to 1, the next time we run the block above, it will do the same for the new tab that has opened. Try it out!

If you ran the compile step with the custom settings, try running the compile step without the custom settings, running `driver.get("http://localhost:8080/")` again, and then rerunning the two blocks above.

# Close down

### Close down the webserver.

`After running, refresh the selenium browser and notice that it can't connect to our website.`

In [ ]:
webserver_process.terminate()

### Close the selenium browser

In [ ]:
driver.quit()

### Remove tempdir

In [ ]:
cleanup_temp_dir()